In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# nice, interactive plotting.
%matplotlib notebook

In [ ]:
# if KSPDIR is set, load the telemetry file from the game directory. Otherwise, load from current directory
if "KSPDIR" in os.environ:
    telemetry_file = os.path.join(os.environ["KSPDIR"], r"GameData\Telemetry\Trajectories.csv")
else:
    telemetry_file = "Trajectories.csv"

In [ ]:
telemetry_file = "./kerbin-flat-nocache_1.7.0rc0.csv"

In [ ]:
data = pd.read_csv(telemetry_file, sep='\t')
data.fillna(method="ffill", inplace=True) # fill everything we can with the previous value

In [ ]:
# shift time axis to start with 0
data["ut"] -= data["ut"][0]

In [ ]:
data.set_index("ut", inplace=True)

In [ ]:
# show all channels for a summary
data.plot(subplots=True, title="Telemetry");

In [ ]:
## arbitrary value, dependent on your trajectory. Please choose appropriately (for example at crash time)
# meas_cutoff = 730
# d = data[((data["altitude"] < 90000) & (data.ut < meas_cutoff))]

In [ ]:
# trim data
# We want the data in the atmosphere, so we clip everything where the air temperature is <= 4K and the airspeed is too small
d = data[((data["temperature"] > 4) & (data["airspeed"] > 100))]

In [ ]:
# High-Altitude Data
d = data[data["altitude"].between(66000, 70000)]

## Angle of Attack

In [ ]:
plt.figure();
aoa_dev = 180 - np.abs(d["aoa"])
plt.plot(aoa_dev)
plt.ylabel("AoA deviation [°]")
plt.xlabel("ut [s]")
plt.title("Deviation from Retrograde");
# plt.ylim([0.0, 0.1])
plt.show();

In [ ]:
np.min(d["force_actual"])

## Forces

In [ ]:
ax = d[["force_actual", "force_predicted", "force_predicted_cache", "altitude"]].plot(secondary_y=["altitude"]);
ax.set_ylabel("force [kN]");
ax.set_ylabel("altitude [m]");
ax.set_xlabel("ut [s]");
plt.title("Forces actual/predicted/cache overview");

In [ ]:
plt.figure();
ax = (d["force_actual"] / d["force_predicted"]).plot()
plt.ylabel("Force ratio [-]")
ax.set_ylim([0.0, 2])
ax.axhline(y=1.0, color='r', linestyle='--')
ax = d["altitude"].plot(ax=ax, secondary_y=True)
# plt.ylabel("Altitude [m]")
plt.xlabel("ut [s]")
plt.xlim([d.index[0], d.index[-1]])
plt.title("Force actual/prediction ratio");
plt.show();

In [ ]:
plt.figure();
ax = (d["force_actual"] - d["force_predicted"]).plot()
plt.ylabel("Force difference [kN]")
ax = d["altitude"].plot(ax=ax, secondary_y=True, legend=True)
ax.set_ylabel("Altitude [m]")
plt.xlabel("ut [s]")
plt.title("Absolute force difference between predicted and actual");
plt.show();

In [ ]:
plt.figure();
ax = ((d["force_actual"] - d["force_predicted"])/d["force_actual"]).plot()
ax.set_ylabel("Force difference [-]")
ax.axhline(y=0.0, color='r', linestyle='--')
ax = d["altitude"].plot(ax=ax, secondary_y=True, legend=True)
ax.set_ylabel("Altitude [m]")
plt.xlabel("ut [s]")
plt.xlim([d.index[0], d.index[-1]])
plt.title("Relative force difference between predicted and actual");
plt.show();

In [ ]:
# this is some math that is not useful and wrong.
# mass = 1.39
# decel_error = np.cumsum(dh["force_actual"] - dh["force_predicted"]) / mass
# np.trapz(y=(dh["airspeed"] - decel_error * np.gradient(dh["ut"])), x=dh["ut"]) / 1000
# np.trapz(y=(decel_error), x=d["ut"]) / 1000

In [ ]:
ax = d[["density", "density_calc", "density_calc_precise", "altitude"]].plot(secondary_y=["altitude"]);
ax.set_ylabel("force [kN]");
ax.set_ylabel("altitude [m]");
ax.set_xlabel("ut [s]");
plt.title("Forces actual/predicted/cache overview");

In [ ]:
plt.figure();
ax = (d["density"] - d["density_calc"]).plot()
plt.ylabel("Density difference [?]")
ax = d["altitude"].plot(ax=ax, secondary_y=True, legend=True)
ax.set_ylabel("Altitude [m]")
plt.xlabel("ut [s]")
plt.title("Absolute density difference between predicted and actual");
plt.show();

In [ ]:
dens_d = (d["density"] - d["density_calc"])